In [34]:
import numpy as np
import pandas as pd
from scipy.stats import poisson, chi2

# --- 1. ЗАВАНТАЖЕННЯ ДАНИХ ---
try:
    df = pd.read_csv('poisson_sample_30_100.csv')
    data = np.array(df['value'].tolist())
except FileNotFoundError:
    # Тестова генерація, якщо файлу немає
    data = np.random.poisson(lam=65, size=1000)
    data = data[(data >= 30) & (data <= 100)]

N = len(data) # n (обсяг вибірки)
alpha = 0.01  # Рівень значущості (як на скріншоті)

print(f"Обсяг вибірки n = {N}")

# --- 2. ОЦІНКА ПАРАМЕТРІВ ---
# Для Пуассона lambda оцінюється як середнє арифметичне (x_bar)
mean_val = np.mean(data)
lambda_hat = mean_val

print(f"Середнє вибіркове (x_bar) = {mean_val:.4f}")
print(f"Оцінка параметра lambda = {lambda_hat:.4f}")
print("-" * 60)

# --- 3. ФОРМУВАННЯ ІНТЕРВАЛІВ (Як на скріншоті) ---
# Створимо інтервали з кроком 5: [30;35), [35;40) ...
step = 5
min_val = 30 # Початок діапазону
max_val = 105 # Кінець діапазону (трохи більше 100)
bins_edges = np.arange(min_val, max_val + step, step)

# Підрахунок емпіричних частот (n_i)
observed_counts, _ = np.histogram(data, bins=bins_edges)

# Розрахунок теоретичних частот (n'_i) для Пуассона
# n' = N * P(потрапляння в інтервал)
expected_counts = []
for i in range(len(bins_edges) - 1):
    low = bins_edges[i]
    high = bins_edges[i+1]

    # Для дискетного Пуассона сумуємо ймовірності всіх цілих чисел в інтервалі [low, high-1]
    k_values = np.arange(low, high)
    prob_interval = np.sum(poisson.pmf(k_values, lambda_hat))

    expected_counts.append(N * prob_interval)

expected_counts = np.array(expected_counts)

# --- 4. ОБ'ЄДНАННЯ МАЛИХ ЧАСТОТ (Алгоритм зі скріншота) ---
# "При цьому об'єднаємо малі частоти... а відповідні їм інтервали об'єднаємо в один"
# Умова: n' має бути >= 5 (класичне правило)

final_obs = []
final_exp = []
final_labels = []

current_obs = 0
current_exp = 0
start_idx = 0

intervals_labels = [f"({bins_edges[i]};{bins_edges[i+1]})" for i in range(len(bins_edges)-1)]

for i in range(len(observed_counts)):
    current_obs += observed_counts[i]
    current_exp += expected_counts[i]

    # Якщо це останній елемент, просто додаємо до поточного накопичення
    is_last = (i == len(observed_counts) - 1)

    if current_exp >= 5 and not is_last:
        final_obs.append(current_obs)
        final_exp.append(current_exp)

        # Формуємо підпис інтервалу
        label_start = bins_edges[start_idx]
        label_end = bins_edges[i+1]
        final_labels.append(f"({label_start};{label_end})")

        current_obs = 0
        current_exp = 0
        start_idx = i + 1

    elif is_last:
        # Обробка останнього елемента (хвоста)
        if len(final_obs) > 0:
            # Якщо хвіст малий, додаємо до попереднього інтервалу
            if current_exp < 5:
                final_obs[-1] += current_obs
                final_exp[-1] += current_exp
                # Оновлюємо підпис попереднього інтервалу
                prev_label_start = final_labels[-1].split(';')[0].replace('(', '')
                final_labels[-1] = f"({prev_label_start};{bins_edges[i+1]})"
            else:
                final_obs.append(current_obs)
                final_exp.append(current_exp)
                label_start = bins_edges[start_idx]
                final_labels.append(f"({label_start};{bins_edges[i+1]})")
        else:
            # Випадок, якщо вся вибірка влізла в одну групу
            final_obs.append(current_obs)
            final_exp.append(current_exp)
            final_labels.append(f"({bins_edges[0]};{bins_edges[-1]})")

final_obs = np.array(final_obs)
final_exp = np.array(final_exp)

# --- 5. ТАБЛИЦЯ РОЗРАХУНКІВ (Як на скріншоті 2) ---
print("\nТаблиця розрахунків критерію:")
print(f"{'Інтервал (x_{{i-1}}; x_i)':<20} | {'n_i':<10} | {'n\'_i':<10} | {'(n_i - n\'_i)^2 / n\'_i':<20}")
print("-" * 70)

chi_terms = []
for i in range(len(final_obs)):
    ni = final_obs[i]
    ni_prime = final_exp[i]
    term = ((ni - ni_prime)**2) / ni_prime
    chi_terms.append(term)
    print(f"{final_labels[i]:<20} | {ni:<10} | {ni_prime:<10.4f} | {term:<20.4f}")

print("-" * 70)
sum_ni = np.sum(final_obs)
sum_ni_prime = np.sum(final_exp)
chi_square_calc = np.sum(chi_terms)
print(f"{'Сума':<20} | {sum_ni:<10} | {sum_ni_prime:<10.4f} | {chi_square_calc:<20.4f}")


# --- 6. ПЕРЕВІРКА ГІПОТЕЗИ ---
# Ступені свободи m = k - 1 - r
# k - кількість інтервалів
# r - кількість оцінених параметрів (у нас 1 - lambda)
k = len(final_obs)
r = 1
dof = k - 1 - r

print("\nДалі знаходимо:")
print(f"Хі-квадрат спостережуване (chi^2_n) = {chi_square_calc:.4f}")
print(f"Число ступенів свободи m = {k} - 1 - {r} = {dof}")

# Критичне значення по таблиці
chi_crit_val = chi2.ppf(1 - alpha, dof)
print(f"За таблицею при alpha = {alpha} та m = {dof} знаходимо c_alpha = {chi_crit_val:.4f}")

# Висновок
print("\nВисновок:")
if chi_square_calc < chi_crit_val:
    print(f"Так як {chi_square_calc:.4f} < {chi_crit_val:.4f}, то гіпотезу про розподіл Пуассона ПРИЙМАЄМО.")
else:
    print(f"Так як {chi_square_calc:.4f} > {chi_crit_val:.4f}, то гіпотезу про розподіл Пуассона ВІДХИЛЯЄМО.")

Обсяг вибірки n = 1000
Середнє вибіркове (x_bar) = 64.7750
Оцінка параметра lambda = 64.7750
------------------------------------------------------------

Таблиця розрахунків критерію:
Інтервал (x_{{i-1}}; x_i) | n_i        | n'_i       | (n_i - n'_i)^2 / n'_i
----------------------------------------------------------------------
(30;50)              | 26         | 25.0385    | 0.0369              
(50;55)              | 87         | 73.2219    | 2.5926              
(55;60)              | 160        | 161.6603   | 0.0171              
(60;65)              | 211        | 234.7205   | 2.3971              
(65;70)              | 232        | 231.3222   | 0.0020              
(70;75)              | 164        | 158.9853   | 0.1582              
(75;80)              | 77         | 78.0116    | 0.0131              
(80;85)              | 32         | 27.8968    | 0.6035              
(85;105)             | 11         | 9.1397     | 0.3786              
--------------------------------------